In [21]:
import numpy as np
import os
import numpy as np
import uncertainties as u
from uncertainties import ufloat
from uncertainties.umath import sqrt
from uncertainties.umath import __all__
from uncertainties.umath import *
import uncertainties.unumpy as unp
import scipy.constants
import os
import matplotlib.pyplot as plt
from scipy.stats import linregress
import pandas # pip install Jinja2
import pint # pip install pint OR conda install pint
units = pint.UnitRegistry()
units.default_format = '~'
import warnings
warnings.simplefilter("ignore")
ulog = u.wrap(np.log)

In [20]:
# rho(Al)
rho=u.UFloat(2.69,0.005) #g/cm^3
N_bg=u.UFloat(73,1/np.sqrt(73)) #anzahl in 120 sekunden
# Import meas 3
X_dicke_meas, t_seconds_meas, N_anzahl_meas = np.genfromtxt(os.path.join(os.getcwd(), "meas_3.csv"), delimiter=',', skip_header=1, unpack=True)

u_x = np.array([u.ufloat(X_val, 5.0) for X_val in X_dicke_meas])
u_t = np.array([u.ufloat(t_val, 2.0/1000) for t_val in t_seconds_meas])
u_N = np.array([u.ufloat(N_val, np.sqrt(N_val)) for N_val in N_anzahl_meas])
#Berechnung N efff
u_Ne = u_N-N_bg
#ln
y_wert=ulog(u_N/u_t)
#Umwandlung in cm
u_x_cm = u_x/10
#rho*x
rho_x = rho * u_x_cm 


def graph_N(X, N, sig_N):
    X, Y = X[2:], N[2:]
    result = scipy.stats.linregress(X, Y)
    #slope, intercept, rvalue, pvalue, sevalue, inteerr
    print("slope: ", result.slope)
    print("slope_err:", result.stderr)
    print("intercept: ", result.intercept)
    x_axis_spaced = np.linspace(np.min(V), np.max(V), 10)
    fig, ax = plt.subplots(1,1,figsize=(12*cm, 12*cm))
    #ax.plot(x,y,linestyle='-', color='red')
    ax.plot(x_axis_spaced, result.slope*x_axis_spaced + result.intercept, 'r--', label='label')
    ax.errorbar(V, N, yerr=sig_N, linestyle='-', color='red', capsize=3, label='errbar')
    ax.set_xlabel('V')
    ax.set_ylabel('N(V)')
    ax.legend(loc='upper left')
    plt.tight_layout()
    plt.savefig('N_of_V', dpi=500)
    plt.show()
    return u.ufloat(result.slope, result.stderr), u.ufloat(result.intercept, result.intercept_stderr) # (slope+=unc), (intercept+=unc)

graph_N(u_x_cm,y_wert , 2)

TypeError: loop of ufunc does not support argument 0 of type AffineScalarFunc which has no callable log method